In [ ]:
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.metrics import classification_report,confusion_matrix,plot_confusion_matrix,make_scorer,accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier 
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_predict,cross_val_score,cross_validate
from sklearn.feature_selection import SelectFromModel
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [ ]:
import pandas as pd
import numpy as np
from numpy import sort
import time
import datetime
from pandas import Series,DataFrame

In [ ]:
import os
os.getcwd()

In [ ]:
Xy1=pd.read_csv("file name.csv")
Xy2016=Xy1.loc[Xy["year"] ==2016]
Xy0515=Xy1.loc[Xy["year"] !=2016]
Xy_0515=Xy0515.drop(columns=["year"])
Xy_2016=Xy2016.drop(columns=["year"])
y_2016=Xy_2016["above_or_below_limit"].copy()
X_2016=Xy_2016.drop(columns=["above_or_below_limit"]).copy()
Xy11=Xy_0515
y=Xy11["above_or_below_limit"].copy()
X=Xy11.drop(columns=["above_or_below_limit",]).copy()

In [ ]:
X=pd.get_dummies(X,columns=['product',"proutgroup","pdg_name_engels","country_of_origin","country_of_analysis","legal_limit"])
X_2016=pd.get_dummies(X_2016,columns=['product',"proutgroup","pdg_name_engels","country_of_origin","country_of_analysis","legal_limit"])
X_2016 = X_2016.reindex(columns = X.columns, fill_value=0)
X.apply(pd.to_numeric)
X_2016.apply(pd.to_numeric)

In [ ]:
X_train1, X_test1, y_train1, y_test1= train_test_split(X,y,train_size=0.9,random_state=1,stratify=y)
sum(y_test1),len(y_test1),sum(y),len(y),sum(y_train1),len(y_train1),
X_train, X_test, y_train, y_test= train_test_split(X_train1,y_train1,train_size=0.9,random_state=1,stratify=y_train1)
scaler = StandardScaler()
scaler.fit(X_train) 
X_train_stander = scaler.transform(X_train)
X_test_stander = scaler.transform(X_test)
X_test1_stander = scaler.transform(X_test1)
X_2016_stander = scaler.transform(X_2016)

In [ ]:
log_clf = LogisticRegression(random_state=42)#,class_weight="balanced",C=0.00001,penalty='l2',)#{0:0.001,1:0.999},max_iter=1000
log_clf.fit(X_train_stander, y_train)
y_pred = log_clf.predict(X_test_stander)
y_pred_train = log_clf.predict(X_train_stander)
y_pred1 = log_clf.predict(X_test1_stander)
y_pred_2016 = log_clf.predict(X_2016_stander)

print(confusion_matrix(y_test, y_pred),roc_auc_score(y_test, y_pred),custom_loss(y_test, y_pred),custom_losshis(y_test, y_pred),recall_score(y_test, y_pred),accuracy_score(y_test, y_pred),f1_score(y_test, y_pred),precision_score(y_test, y_pred))
print(confusion_matrix(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train),custom_loss(y_train, y_pred_train),custom_losshis(y_train, y_pred_train),recall_score(y_train, y_pred_train),accuracy_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),precision_score(y_train, y_pred_train))
print(confusion_matrix(y_test1, y_pred1),roc_auc_score(y_test1, y_pred1),custom_loss(y_test1, y_pred1),custom_losshis(y_test1, y_pred1),recall_score(y_test1, y_pred1),accuracy_score(y_test1, y_pred1),f1_score(y_test1, y_pred1),precision_score(y_test1, y_pred1))
print(confusion_matrix(y_2016, y_pred_2016),roc_auc_score(y_2016, y_pred_2016),custom_loss(y_2016, y_pred_2016),custom_losshis(y_2016, y_pred_2016),recall_score(y_2016, y_pred_2016),accuracy_score(y_2016, y_pred_2016),f1_score(y_2016, y_pred_2016),precision_score(y_2016, y_pred_2016))

In [ ]:
result_log = {'AUC_train':[roc_auc_score(y_train, y_pred_train)],"AUC_validation":[roc_auc_score(y_test, y_pred)],'AUC_test':[roc_auc_score(y_test1, y_pred1)],'AUC_2016':[roc_auc_score(y_2016, y_pred_2016)],                                                                                                          
              'Recall_train':[recall_score(y_train, y_pred_train)],"Recall_validation":[recall_score(y_test, y_pred)],'Recall_test':[recall_score(y_test1, y_pred1)],'Recall_2016':[recall_score(y_2016, y_pred_2016)],                                                                                                               
              'Precision_train':[precision_score(y_train, y_pred_train)],"Precision_validation":[precision_score(y_test, y_pred)],'Precision_test':[precision_score(y_test1, y_pred1)],'Precision_2016':[precision_score(y_2016, y_pred_2016)],                                                                                                         
              'Accuracy_train':[accuracy_score(y_train, y_pred_train)],"Accuracy_validation":[accuracy_score(y_test, y_pred)],'Accuracy_test':[accuracy_score(y_test1, y_pred1)],'Accuracy_2016':[accuracy_score(y_2016, y_pred_2016)],                                                                                                     
             'F1_train':[f1_score(y_train, y_pred_train)],"F1_validation":[f1_score(y_test, y_pred)],'F1_test':[f1_score(y_test1, y_pred1)],'F1_2016':[f1_score(y_2016, y_pred_2016)],
          'Custom_train':[custom_loss(y_train, y_pred_train)],"Custom_validation":[custom_loss(y_test, y_pred)], 'Custom_test':[custom_loss(y_test1, y_pred1)],'Custom_2016':[custom_loss(y_2016, y_pred_2016)],'Customhis_train':[custom_losshis(y_train, y_pred_train)],"Customhis_validation":[custom_losshis(y_test, y_pred)], 'Custom_test':[custom_losshis(y_test1, y_pred1)],'Custom_2016':[custom_loss(y_2016, y_pred_2016)],                                                                                                          
        'Confusion_matrix_train':[confusion_matrix(y_train, y_pred_train).flatten()],'Confusion_matrix_test':[confusion_matrix(y_test, y_pred).flatten()],'Confusion_matrix_fianl':[confusion_matrix(y_test1, y_pred1).flatten()],'Confusion_matrix_2016':[confusion_matrix(y_2016, y_pred_2016).flatten()]}
result_log = DataFrame(result_log)

In [ ]:
sgd_svm=SGDClassifier(random_state=42,loss='hinge')#,loss='hinge', n_iter_no_change=3,validation_fraction=0.1,class_weight="balanced",early_stopping=True,max_iter=1000, tol=1e-3)
#CV
sgd_svm.fit(X_train_stander, y_train)
y_pred = sgd_svm.predict(X_test_stander)
y_pred_train = sgd_svm.predict(X_train_stander)
y_pred1 = sgd_svm.predict(X_test1_stander)
#y_pred_2015 = sgd_svm.predict(X_2015_stander)
y_pred_2016 = sgd_svm.predict(X_2016_stander)

print(confusion_matrix(y_test, y_pred),roc_auc_score(y_test, y_pred),custom_loss(y_test, y_pred),recall_score(y_test, y_pred),accuracy_score(y_test, y_pred),f1_score(y_test, y_pred),precision_score(y_test, y_pred))
print(confusion_matrix(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train),custom_loss(y_train, y_pred_train),recall_score(y_train, y_pred_train),accuracy_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),precision_score(y_train, y_pred_train))
print(confusion_matrix(y_test1, y_pred1),roc_auc_score(y_test1, y_pred1),custom_loss(y_test1, y_pred1),recall_score(y_test1, y_pred1),accuracy_score(y_test1, y_pred1),f1_score(y_test1, y_pred1),precision_score(y_test1, y_pred1))
#print(confusion_matrix(y_2015, y_pred_2015),roc_auc_score(y_2015, y_pred_2015),custom_loss(y_2015, y_pred_2015),recall_score(y_2015, y_pred_2015),accuracy_score(y_2015, y_pred_2015),f1_score(y_2015, y_pred_2015),precision_score(y_2015, y_pred_2015))
print(confusion_matrix(y_2016, y_pred_2016),roc_auc_score(y_2016, y_pred_2016),custom_loss(y_2016, y_pred_2016),
      recall_score(y_2016, y_pred_2016),accuracy_score(y_2016, y_pred_2016),f1_score(y_2016, y_pred_2016),precision_score(y_2016, y_pred_2016))

In [ ]:
result_svm = {'AUC_train':[roc_auc_score(y_train, y_pred_train)],"AUC_validation":[roc_auc_score(y_test, y_pred)],'AUC_test':[roc_auc_score(y_test1, y_pred1)],'AUC_2016':[roc_auc_score(y_2016, y_pred_2016)],                                                                                                          
              'Recall_train':[recall_score(y_train, y_pred_train)],"Recall_validation":[recall_score(y_test, y_pred)],'Recall_test':[recall_score(y_test1, y_pred1)],'Recall_2016':[recall_score(y_2016, y_pred_2016)],                                                                                                               
              'Precision_train':[precision_score(y_train, y_pred_train)],"Precision_validation":[precision_score(y_test, y_pred)],'Precision_test':[precision_score(y_test1, y_pred1)],'Precision_2016':[precision_score(y_2016, y_pred_2016)],                                                                                                         
              'Accuracy_train':[accuracy_score(y_train, y_pred_train)],"Accuracy_validation":[accuracy_score(y_test, y_pred)],'Accuracy_test':[accuracy_score(y_test1, y_pred1)],'Accuracy_2016':[accuracy_score(y_2016, y_pred_2016)],                                                                                                     
             'F1_train':[f1_score(y_train, y_pred_train)],"F1_validation":[f1_score(y_test, y_pred)],'F1_test':[f1_score(y_test1, y_pred1)],'F1_2016':[f1_score(y_2016, y_pred_2016)],
          'Custom_train':[custom_loss(y_train, y_pred_train)],"Custom_validation":[custom_loss(y_test, y_pred)], 'Custom_test':[custom_loss(y_test1, y_pred1)],'Custom_2016':[custom_loss(y_2016, y_pred_2016)],'Customhis_train':[custom_losshis(y_train, y_pred_train)],"Customhis_validation":[custom_losshis(y_test, y_pred)], 'Custom_test':[custom_losshis(y_test1, y_pred1)],'Custom_2016':[custom_loss(y_2016, y_pred_2016)],                                                                                                          
        'Confusion_matrix_train':[confusion_matrix(y_train, y_pred_train).flatten()],'Confusion_matrix_test':[confusion_matrix(y_test, y_pred).flatten()],'Confusion_matrix_fianl':[confusion_matrix(y_test1, y_pred1).flatten()],'Confusion_matrix_2016':[confusion_matrix(y_2016, y_pred_2016).flatten()]}
result_svm = DataFrame(result_svm)

In [ ]:
dt_clf = DecisionTreeClassifier(random_state=42)#,class_weight="balanced",max_depth=10,min_samples_leaf=3)
dt_clf.fit(X_train_stander, y_train)
y_pred = dt_clf.predict(X_test_stander)
y_pred_train = dt_clf.predict(X_train_stander)
y_pred1 = dt_clf.predict(X_test1_stander)
y_pred_2016 = dt_clf.predict(X_2016_stander)

print(confusion_matrix(y_test, y_pred),roc_auc_score(y_test, y_pred),custom_loss(y_test, y_pred),recall_score(y_test, y_pred),accuracy_score(y_test, y_pred),f1_score(y_test, y_pred),precision_score(y_test, y_pred))
print(confusion_matrix(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train),custom_loss(y_train, y_pred_train),recall_score(y_train, y_pred_train),accuracy_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),precision_score(y_train, y_pred_train))
print(confusion_matrix(y_test1, y_pred1),roc_auc_score(y_test1, y_pred1),custom_loss(y_test1, y_pred1),recall_score(y_test1, y_pred1),accuracy_score(y_test1, y_pred1),f1_score(y_test1, y_pred1),precision_score(y_test1, y_pred1))
print(confusion_matrix(y_2016, y_pred_2016),roc_auc_score(y_2016, y_pred_2016),custom_loss(y_2016, y_pred_2016),recall_score(y_2016, y_pred_2016),accuracy_score(y_2016, y_pred_2016),f1_score(y_2016, y_pred_2016),precision_score(y_2016, y_pred_2016))
print(custom_losshis(y_2016, y_pred_2016))

In [ ]:
result_dt = {'AUC_train':[roc_auc_score(y_train, y_pred_train)],"AUC_validation":[roc_auc_score(y_test, y_pred)],'AUC_test':[roc_auc_score(y_test1, y_pred1)],'AUC_2016':[roc_auc_score(y_2016, y_pred_2016)],                                                                                                          
              'Recall_train':[recall_score(y_train, y_pred_train)],"Recall_validation":[recall_score(y_test, y_pred)],'Recall_test':[recall_score(y_test1, y_pred1)],'Recall_2016':[recall_score(y_2016, y_pred_2016)],                                                                                                               
              'Precision_train':[precision_score(y_train, y_pred_train)],"Precision_validation":[precision_score(y_test, y_pred)],'Precision_test':[precision_score(y_test1, y_pred1)],'Precision_2016':[precision_score(y_2016, y_pred_2016)],                                                                                                         
              'Accuracy_train':[accuracy_score(y_train, y_pred_train)],"Accuracy_validation":[accuracy_score(y_test, y_pred)],'Accuracy_test':[accuracy_score(y_test1, y_pred1)],'Accuracy_2016':[accuracy_score(y_2016, y_pred_2016)],                                                                                                     
             'F1_train':[f1_score(y_train, y_pred_train)],"F1_validation":[f1_score(y_test, y_pred)],'F1_test':[f1_score(y_test1, y_pred1)],'F1_2016':[f1_score(y_2016, y_pred_2016)],
          'Custom_train':[custom_loss(y_train, y_pred_train)],"Custom_validation":[custom_loss(y_test, y_pred)], 'Custom_test':[custom_loss(y_test1, y_pred1)],'Custom_2016':[custom_loss(y_2016, y_pred_2016)],'Customhis_train':[custom_losshis(y_train, y_pred_train)],"Customhis_validation":[custom_losshis(y_test, y_pred)], 'Custom_test':[custom_losshis(y_test1, y_pred1)],'Custom_2016':[custom_loss(y_2016, y_pred_2016)],                                                                                                          
        'Confusion_matrix_train':[confusion_matrix(y_train, y_pred_train).flatten()],'Confusion_matrix_test':[confusion_matrix(y_test, y_pred).flatten()],'Confusion_matrix_fianl':[confusion_matrix(y_test1, y_pred1).flatten()],'Confusion_matrix_2016':[confusion_matrix(y_2016, y_pred_2016).flatten()]}
result_dt = DataFrame(result_dt)

In [ ]:
xgb_clf = xgb.XGBClassifier(random_state=42)#,objective="binary:logistic",gamma=1.0, learning_rate=0.1, max_depth=4,reg_lambda=10.0,scale_pos_weight=559,subsample=0.5)#subsample=0.9, scale_pos_weight=1000,max_depth=4)#,objective="binary:logistic",gamma=1.0, learning_rate=0.1, max_depth=4,reg_lambda=10.0,scale_pos_weight=5,subsample=0.5)#subsample=0.9, scale_pos_weight=1000,max_depth=4
xgb_clf.fit(X_train_stander, y_train)#,eval_set=eval_set, eval_metric=['error',"logloss","aucpr","auc",],early_stopping_rounds=20)
y_pred = xgb_clf.predict(X_test_stander)
y_pred_train = xgb_clf.predict(X_train_stander)
y_pred1 = xgb_clf.predict(X_test1_stander)
y_pred_2016 = xgb_clf.predict(X_2016_stander)

print(confusion_matrix(y_test, y_pred),roc_auc_score(y_test, y_pred),custom_loss(y_test, y_pred),recall_score(y_test, y_pred),accuracy_score(y_test, y_pred),f1_score(y_test, y_pred),precision_score(y_test, y_pred))
print(confusion_matrix(y_train, y_pred_train),roc_auc_score(y_train, y_pred_train),custom_loss(y_train, y_pred_train),recall_score(y_train, y_pred_train),accuracy_score(y_train, y_pred_train),f1_score(y_train, y_pred_train),precision_score(y_train, y_pred_train))
print(confusion_matrix(y_test1, y_pred1),roc_auc_score(y_test1, y_pred1),custom_loss(y_test1, y_pred1),recall_score(y_test1, y_pred1),accuracy_score(y_test1, y_pred1),f1_score(y_test1, y_pred1),precision_score(y_test1, y_pred1))
print(confusion_matrix(y_2016, y_pred_2016),roc_auc_score(y_2016, y_pred_2016),custom_loss(y_2016, y_pred_2016),
      recall_score(y_2016, y_pred_2016),accuracy_score(y_2016, y_pred_2016),f1_score(y_2016, y_pred_2016),precision_score(y_2016, y_pred_2016))


In [ ]:
result_xgb = {'AUC_train':[roc_auc_score(y_train, y_pred_train)],"AUC_validation":[roc_auc_score(y_test, y_pred)],'AUC_test':[roc_auc_score(y_test1, y_pred1)],'AUC_2016':[roc_auc_score(y_2016, y_pred_2016)],                                                                                                          
              'Recall_train':[recall_score(y_train, y_pred_train)],"Recall_validation":[recall_score(y_test, y_pred)],'Recall_test':[recall_score(y_test1, y_pred1)],'Recall_2016':[recall_score(y_2016, y_pred_2016)],                                                                                                               
              'Precision_train':[precision_score(y_train, y_pred_train)],"Precision_validation":[precision_score(y_test, y_pred)],'Precision_test':[precision_score(y_test1, y_pred1)],'Precision_2016':[precision_score(y_2016, y_pred_2016)],                                                                                                         
              'Accuracy_train':[accuracy_score(y_train, y_pred_train)],"Accuracy_validation":[accuracy_score(y_test, y_pred)],'Accuracy_test':[accuracy_score(y_test1, y_pred1)],'Accuracy_2016':[accuracy_score(y_2016, y_pred_2016)],                                                                                                     
             'F1_train':[f1_score(y_train, y_pred_train)],"F1_validation":[f1_score(y_test, y_pred)],'F1_test':[f1_score(y_test1, y_pred1)],'F1_2016':[f1_score(y_2016, y_pred_2016)],
          'Custom_train':[custom_loss(y_train, y_pred_train)],"Custom_validation":[custom_loss(y_test, y_pred)], 'Custom_test':[custom_loss(y_test1, y_pred1)],'Custom_2016':[custom_loss(y_2016, y_pred_2016)],'Customhis_train':[custom_losshis(y_train, y_pred_train)],"Customhis_validation":[custom_losshis(y_test, y_pred)], 'Custom_test':[custom_losshis(y_test1, y_pred1)],'Custom_2016':[custom_loss(y_2016, y_pred_2016)],                                                                                                          
        'Confusion_matrix_train':[confusion_matrix(y_train, y_pred_train).flatten()],'Confusion_matrix_test':[confusion_matrix(y_test, y_pred).flatten()],'Confusion_matrix_fianl':[confusion_matrix(y_test1, y_pred1).flatten()],'Confusion_matrix_2016':[confusion_matrix(y_2016, y_pred_2016).flatten()]}
result_xgb = DataFrame(result_xgb)

In [ ]:
result_tuning= pd.concat([result_log, result_svm, result_dt,result_xgb],names=['Model'],keys=['result_log', 'result_svm', 'result_dt',"result_xgb"])
result_tuning.to_csv("result_untuning_afb1.csv")
result_tuning